In [1]:
import numpy as np
import arviz as az
from line_profiler import LineProfiler
from arviz.data import load_arviz_data,from_dict,convert_to_inference_data
from arviz.stats.stats import r2_score, hpd, _gpdfit, _gpinv,_ic_matrix, waic,psislw
from arviz.stats.stats_utils import logsumexp as _logsumexp
import scipy.stats as st
import numba
import math
from numpy import sin, cos
import pandas as pd
import warnings

In [2]:
data_1 = np.random.randn(10_000,1000)
data_2 = np.random.randn(1_000_000)
school = load_arviz_data("centered_eight").posterior["mu"].values

In [3]:
'''********************************************hpd********************************************************************'''

'********************************************hpd********************************************************************'

In [4]:
lp = LineProfiler()
wrapper = lp(hpd)
wrapper(data_1, 0.94, True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 2.36279 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: hpd at line 252

Line #      Hits         Time  Per Hit   % Time  Line Contents
   252                                           def hpd(ary, credible_interval=0.94, circular=False):
   253                                               """
   254                                               Calculate highest posterior density (HPD) of array for given credible_interval.
   255                                           
   256                                               The HPD is the minimum width Bayesian credible interval (BCI). This implementation works only
   257                                               for unimodal distributions.
   258                                           
   259                                               Parameters
   260                                               ----------
   261                                               x : Numpy a

In [5]:
lp = LineProfiler()
wrapper = lp(hpd)
wrapper(data_2, 0.94, True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.591736 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: hpd at line 252

Line #      Hits         Time  Per Hit   % Time  Line Contents
   252                                           def hpd(ary, credible_interval=0.94, circular=False):
   253                                               """
   254                                               Calculate highest posterior density (HPD) of array for given credible_interval.
   255                                           
   256                                               The HPD is the minimum width Bayesian credible interval (BCI). This implementation works only
   257                                               for unimodal distributions.
   258                                           
   259                                               Parameters
   260                                               ----------
   261                                               x : Numpy 

In [6]:
lp = LineProfiler()
wrapper = lp(hpd)
wrapper(school, 0.94, True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.120716 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: hpd at line 252

Line #      Hits         Time  Per Hit   % Time  Line Contents
   252                                           def hpd(ary, credible_interval=0.94, circular=False):
   253                                               """
   254                                               Calculate highest posterior density (HPD) of array for given credible_interval.
   255                                           
   256                                               The HPD is the minimum width Bayesian credible interval (BCI). This implementation works only
   257                                               for unimodal distributions.
   258                                           
   259                                               Parameters
   260                                               ----------
   261                                               x : Numpy 

In [7]:
'''
Bottlenecks:
    1)scipy.stats.circmean
    2)numpy arctan2
'''

'\nBottlenecks:\n    1)scipy.stats.circmean\n    2)numpy arctan2\n'

In [8]:
def _circ_mean(ary, high, low):
    ary = np.asarray(ary)
    if ary.size==0:
        return np.nan, np.nan
    pi = np.pi
    angles = (ary-low)*2*pi/(high-low)
    S = sinusoidal(angles)
    C = cosine(angles)
    res = np.arctan2(S,C)
    mask = res < 0
    if mask.ndim > 0:
        res[mask] += 2*pi
    elif mask:
        res += 2*pi
    return res*(high - low)/2.0/pi + low

@numba.jit(nopython=True)
def sinusoidal(x):
    summ = 0
    x = x.flatten()
    for i in range(0,len(x)):
        summ = summ+math.sin(x[i])
    return summ
@numba.jit(nopython=True)
def cosine(x):
    summ = 0
    x = x.flatten()
    for i in range(0,len(x)):
        summ = summ+math.cos(x[i])
    return summ


In [159]:
%timeit sinusoidal(data_1)

33.5 ms ± 181 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [158]:
%timeit np.sin(data_1)

2.67 ms ± 102 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [160]:
%timeit sinusoidal(data_2)

32.8 ms ± 296 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [161]:
%timeit np.sin(data_2)

2.58 ms ± 89.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%timeit cosine(data_1)

372 ms ± 2.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%timeit np.cos(data_1)

51.3 ms ± 746 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit cosine(data_2)

32.3 ms ± 900 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit np.cos(data_2)

2.59 ms ± 81.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
np.allclose(_circ_mean(data_1, np.pi, -np.pi), st.circmean(data_1, np.pi, -np.pi))

True

In [18]:
np.allclose(_circ_mean(data_2, np.pi, -np.pi), st.circmean(data_2, np.pi, -np.pi))

True

In [19]:
np.allclose(_circ_mean(school, np.pi, -np.pi), st.circmean(school, np.pi, -np.pi))

True

In [20]:
%timeit _circ_mean(data_1, np.pi, -np.pi)

1.02 s ± 7.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%timeit st.circmean(data_1, np.pi, -np.pi)

213 ms ± 4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit _circ_mean(data_2, np.pi, -np.pi)

96.9 ms ± 453 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%timeit st.circmean(data_2, np.pi, -np.pi)

19.1 ms ± 364 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit _circ_mean(school, np.pi, -np.pi)

186 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [25]:
%timeit st.circmean(school, np.pi, -np.pi)

207 µs ± 3.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [162]:
'''Performance of _circ_mean is a worse on larger datasets.
   On schools, the performance is much better.
   ¯\_(ツ)_/¯
   Let's see the overall perfomance of hpd
'''

"Performance of _circ_mean is a worse on larger datasets.\n   On schools, the performance is much better.\n   ¯\\_(ツ)_/¯\n   Let's see the overall perfomance of hpd\n"

In [27]:
def hpd_new(ary, credible_interval=0.94, circular=False):
    if ary.ndim > 1:
        hpd_array = np.array(
            [hpd(row, credible_interval=credible_interval, circular=circular) for row in ary.T]
        )
        return hpd_array
    # Make a copy of trace
    ary = ary.copy()
    n = len(ary)

    if circular:
        mean = _circ_mean(ary, high=np.pi, low=-np.pi)
        ary = ary - mean
        ary = np.arctan2(np.sin(ary), np.cos(ary))

    ary = np.sort(ary)
    interval_idx_inc = int(np.floor(credible_interval * n))
    n_intervals = n - interval_idx_inc
    interval_width = ary[interval_idx_inc:] - ary[:n_intervals]

    if len(interval_width) == 0:
        raise ValueError(
            "Too few elements for interval calculation. "
            "Check that credible_interval meets condition 0 =< credible_interval < 1"
        )

    min_idx = np.argmin(interval_width)
    hdi_min = ary[min_idx]
    hdi_max = ary[min_idx + interval_idx_inc]

    if circular:
        hdi_min = hdi_min + mean
        hdi_max = hdi_max + mean
        hdi_min = np.arctan2(np.sin(hdi_min), np.cos(hdi_min))
        hdi_max = np.arctan2(np.sin(hdi_max), np.cos(hdi_max))

    return np.array([hdi_min, hdi_max])

In [28]:
np.allclose(hpd_new(school, 0.95,True), az.stats.hpd(school, 0.95,True))

True

In [29]:
np.allclose(hpd_new(data_1, 0.95,True), az.stats.hpd(data_1, 0.95,True))

True

In [30]:
np.allclose(hpd_new(data_2, 0.95,True), az.stats.hpd(data_2, 0.95,True))

True

In [31]:
%timeit hpd_new(school, 0.95,True)

37.1 ms ± 672 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit az.stats.hpd(school, 0.95,True)

36.7 ms ± 418 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit hpd_new(data_1, 0.95,True)

1.96 s ± 13.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
%timeit az.stats.hpd(data_1, 0.95,True)

1.96 s ± 6.56 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit hpd_new(data_2, 0.95,True)

312 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%timeit az.stats.hpd(data_2, 0.95,True)

235 ms ± 2.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
'''As seen above, the performance of numba is better with school dataset.
The performance with large datasets is a bit unstable.'''

'As seen above, the performance of numba is better with school dataset.\nThe performance with large datasets is a bit unstable.'

In [38]:
'''******************************************R2_SCORE***************************************************************'''

'******************************************R2_SCORE***************************************************************'

In [39]:
data_1 = np.random.randn(1000,1000)
data_2 = np.random.randn(1_000_000)
data_3 = np.random.randn(1000,1000)
data_4 = np.random.randn(1_000_000)
school = load_arviz_data("centered_eight").posterior["mu"].values
n_school = load_arviz_data("non_centered_eight").posterior["mu"].values

In [40]:
lp = LineProfiler()
wrapper = lp(r2_score)
wrapper(data_1, data_3)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.198642 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: r2_score at line 568

Line #      Hits         Time  Per Hit   % Time  Line Contents
   568                                           def r2_score(y_true, y_pred):
   569                                               """R² for Bayesian regression models. Only valid for linear models.
   570                                           
   571                                               Parameters
   572                                               ----------
   573                                               y_true: : array-like of shape = (n_samples) or (n_samples, n_outputs)
   574                                                   Ground truth (correct) target values.
   575                                               y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
   576                                                   Estimated target values.
   577 

In [41]:
lp = LineProfiler()
wrapper = lp(r2_score)
wrapper(data_2, data_4)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.024834 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: r2_score at line 568

Line #      Hits         Time  Per Hit   % Time  Line Contents
   568                                           def r2_score(y_true, y_pred):
   569                                               """R² for Bayesian regression models. Only valid for linear models.
   570                                           
   571                                               Parameters
   572                                               ----------
   573                                               y_true: : array-like of shape = (n_samples) or (n_samples, n_outputs)
   574                                                   Ground truth (correct) target values.
   575                                               y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
   576                                                   Estimated target values.
   577 

In [42]:
@numba.njit
def _var_1d(data):
    a,b = 0,0
    for i in data:
        a = a+i
        b = b+i*i
    return b/len(data)-((a/len(data))**2)

@numba.jit
def _var_2d(data):
    a,b = data.shape
    var = np.zeros(b)
    for i in range(0,b):
        var[i] = _var_1d(data[:,i])
    return var

In [43]:
_var_1d(data_2)

0.9979012160953333

In [44]:
np.var(data_2)

0.997901216095329

In [45]:
np.allclose(_var_1d(data_2), np.var(data_2))

True

In [46]:
np.allclose(_var_1d(data_4), np.var(data_4))

True

In [47]:
%timeit _var_1d(data_2-data_4)

4.65 ms ± 55.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
%timeit np.var(data_2-data_4)

10.9 ms ± 62.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
np.allclose(_var_1d(data_2-data_4),np.var(data_2-data_4))

True

In [50]:
%timeit _var_2d(school)

46.6 µs ± 4.07 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%timeit np.var(school,0)

51.8 µs ± 626 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [52]:
%timeit _var_2d(data_1)

3.04 ms ± 39 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
%timeit np.var(data_1,0)

6.58 ms ± 79.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
%timeit _var_2d(data_3-data_1)

6.02 ms ± 52.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
%timeit np.var(data_3-data_1,0)

14.6 ms ± 30.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [56]:
np.allclose(_var_2d(school), np.var(school,0))

True

In [57]:
np.allclose(_var_2d(data_1), np.var(data_1,0))

True

In [58]:
np.allclose(_var_2d(data_3), np.var(data_3,0))

True

In [59]:
np.allclose(_var_2d(data_3-data_1), np.var(data_3-data_1,0))

True

In [60]:
np.allclose(_var_2d(data_1-data_3), np.var(data_1-data_3,0))

True

In [61]:
'''Numba is doing wonders with variance. Lets inspect mean'''

'Numba is doing wonders with variance. Lets inspect mean'

In [62]:
@numba.njit
def mean_1d(data):
    summ = 0
    for i in data:
        summ = summ+i
    return summ/len(data)

In [63]:
mean_1d(data_2-data_4)

0.0001399785322285794

In [64]:
np.mean(data_2-data_4)

0.00013997853222859263

In [65]:
%timeit mean_1d(data_2-data_4)


4.63 ms ± 60.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
%timeit np.mean(data_2-data_4)

3.89 ms ± 56.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [67]:
'''Numba is not suitable here. Lets see the overall r2_score performance'''

'Numba is not suitable here. Lets see the overall r2_score performance'

In [68]:
def r2_score_new(y_true, y_pred):
    if y_pred.ndim == 1:
        var_y_est = _var_1d(y_pred)
        var_e = _var_1d(y_true - y_pred)
    else:
        var_y_est = _var_1d(y_pred.mean(0))
        var_e = _var_2d(y_true - y_pred)

    r_squared = var_y_est / (var_y_est + var_e)

    return pd.Series([np.mean(r_squared), np.std(r_squared)], index=["r2", "r2_std"])

In [69]:
%timeit r2_score_new(data_2, data_4)

7.03 ms ± 44.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [70]:
%timeit az.stats.r2_score(data_2, data_4)

17.9 ms ± 443 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%timeit r2_score_new(data_1, data_3)

7.98 ms ± 67.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [72]:
%timeit az.stats.r2_score(data_1, data_3)

17.6 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [73]:
%timeit r2_score_new(school, n_school)

575 µs ± 5.77 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [74]:
%timeit az.stats.r2_score(school, n_school)

633 µs ± 6.15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [75]:
df_1 = r2_score_new(school, n_school)
df_2 = az.stats.r2_score(school, n_school)

In [76]:
df_1

r2        0.219553
r2_std    0.168670
dtype: float64

In [77]:
df_2

r2        0.219553
r2_std    0.168670
dtype: float64

In [78]:
df_1==df_2

r2        False
r2_std    False
dtype: bool

In [79]:
r2_score_new(data_1, data_3)

r2        0.000502
r2_std    0.000023
dtype: float64

In [80]:
az.stats.r2_score(data_1, data_3)

r2        0.000502
r2_std    0.000023
dtype: float64

In [81]:
'''I have not included the pandas test yet...will include them soon'''

'I have not included the pandas test yet...will include them soon'

In [82]:
'''It can be clearly seen that numba greatly improves the performance of r2_score.
TBH, this was the function which I thought had the least scope of improvement
Thank you mentors.'''

'It can be clearly seen that numba greatly improves the performance of r2_score.\nTBH, this was the function which I thought had the least scope of improvement\nThank you mentors.'

In [83]:
"""""""""""""""""""""""""""""""""""""""""""_gpdfit"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""


'_gpdfit'

In [84]:
data = np.abs(np.sort(np.random.randn(100000)))
school = np.abs(np.sort((load_arviz_data("centered_eight").posterior["mu"].values)[1,:]))

In [85]:
lp = LineProfiler()
wrapper = lp(_gpdfit)
wrapper(data)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.479742 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _gpdfit at line 491

Line #      Hits         Time  Per Hit   % Time  Line Contents
   491                                           def _gpdfit(ary):
   492                                               """Estimate the parameters for the Generalized Pareto Distribution (GPD).
   493                                           
   494                                               Empirical Bayes estimate for the parameters of the generalized Pareto
   495                                               distribution given the data.
   496                                           
   497                                               Parameters
   498                                               ----------
   499                                               ary : array
   500                                                   sorted 1D data array
   501                                

In [86]:
lp = LineProfiler()
wrapper = lp(_gpdfit)
wrapper(school)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.001702 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _gpdfit at line 491

Line #      Hits         Time  Per Hit   % Time  Line Contents
   491                                           def _gpdfit(ary):
   492                                               """Estimate the parameters for the Generalized Pareto Distribution (GPD).
   493                                           
   494                                               Empirical Bayes estimate for the parameters of the generalized Pareto
   495                                               distribution given the data.
   496                                           
   497                                               Parameters
   498                                               ----------
   499                                               ary : array
   500                                                   sorted 1D data array
   501                                

In [87]:
"""Bottleneck at 518"""

'Bottleneck at 518'

In [88]:
@numba.njit
def log_1p(data):
    log_p = np.zeros_like(data)
    for i in range(0,len(data)):
        log_p[i] = math.log1p(data[i])
    return log_p

In [89]:
np.allclose(log_1p(data), np.log1p(data))

True

In [90]:
np.allclose(log_1p(school), np.log1p(school))

True

In [91]:
%timeit log_1p(data)

1.72 ms ± 27.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [92]:
%timeit np.log1p(data)

251 µs ± 5.56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [93]:
%timeit log_1p(school)

10.6 µs ± 27.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [94]:
%timeit np.log1p(school)

10.7 µs ± 31.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [95]:
'''Performance is somewhat unstable, almost half the time it speeds up the process, sometimes its performance is
while in 10% of trials the performance comes down a notch w.r.t to numpy'''

'Performance is somewhat unstable, almost half the time it speeds up the process, sometimes its performance is\nwhile in 10% of trials the performance comes down a notch w.r.t to numpy'

In [96]:
'''overall performance of gpdfit'''

'overall performance of gpdfit'

In [97]:
def _gpdfit_new(ary):
    prior_bs = 3
    prior_k = 10
    n = len(ary)
    m_est = 30 + int(n ** 0.5)

    b_ary = 1 - np.sqrt(m_est / (np.arange(1, m_est + 1, dtype=float) - 0.5))
    b_ary /= prior_bs * ary[int(n / 4 + 0.5) - 1]
    b_ary += 1 / ary[-1]

    k_ary = np.log1p(-b_ary[:, None] * ary).mean(axis=1) # pylint: disable=no-member Using my own log_1p here throws an exception
    len_scale = n * (np.log(-(b_ary / k_ary)) - k_ary - 1)
    weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)

    # remove negligible weights
    real_idxs = weights >= 10 * np.finfo(float).eps
    if not np.all(real_idxs):
        weights = weights[real_idxs]
        b_ary = b_ary[real_idxs]
    # normalise weights
    weights /= weights.sum()

    # posterior mean for b
    b_post = np.sum(b_ary * weights)
    # estimate for k
    k_post = log_1p(-b_post * ary).mean()  # pylint: disable=invalid-unary-operand-type,no-member
    # add prior for k_post
    k_post = (n * k_post + prior_k * 0.5) / (n + prior_k)
    sigma = -k_post / b_post
    return k_post, sigma

In [98]:
%timeit _gpdfit_new(data)

413 ms ± 5.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [99]:
%timeit az.stats.stats._gpdfit(data)

410 ms ± 4.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [100]:
%timeit _gpdfit_new(school)

398 µs ± 7.52 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [101]:
%timeit az.stats.stats._gpdfit(school)

393 µs ± 5.59 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [102]:
'''Similar performance on both the datasets. Up for discussion with mentors'''

'Similar performance on both the datasets. Up for discussion with mentors'

In [103]:
"""""""""""""""""""""""""""""""""""""""""""""_gpvinv"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'_gpvinv'

In [104]:
data = np.random.randn(100000)
school = load_arviz_data("centered_eight").posterior["mu"].values[1,:]

In [105]:
lp = LineProfiler()
wrapper = lp(_gpinv)
wrapper(data, 5,2)
lp.print_stats()


Timer unit: 1e-06 s

Total time: 0.004106 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _gpinv at line 541

Line #      Hits         Time  Per Hit   % Time  Line Contents
   541                                           def _gpinv(probs, kappa, sigma):
   542                                               """Inverse Generalized Pareto distribution function."""
   543                                               # pylint: disable=unsupported-assignment-operation, invalid-unary-operand-type
   544         1        908.0    908.0     22.1      x = np.full_like(probs, np.nan)
   545         1          5.0      5.0      0.1      if sigma <= 0:
   546                                                   return x
   547         1        509.0    509.0     12.4      ok = (probs > 0) & (probs < 1)
   548         1         44.0     44.0      1.1      if np.all(ok):
   549                                                   if np.abs(kappa) < np.finfo(float).eps:
   550            

In [106]:
lp = LineProfiler()
wrapper = lp(_gpinv)
wrapper(school, 5,2)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.000195 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _gpinv at line 541

Line #      Hits         Time  Per Hit   % Time  Line Contents
   541                                           def _gpinv(probs, kappa, sigma):
   542                                               """Inverse Generalized Pareto distribution function."""
   543                                               # pylint: disable=unsupported-assignment-operation, invalid-unary-operand-type
   544         1         32.0     32.0     16.4      x = np.full_like(probs, np.nan)
   545         1          2.0      2.0      1.0      if sigma <= 0:
   546                                                   return x
   547         1         27.0     27.0     13.8      ok = (probs > 0) & (probs < 1)
   548         1         30.0     30.0     15.4      if np.all(ok):
   549                                                   if np.abs(kappa) < np.finfo(float).eps:
   550            

In [107]:
'''Log1p or expm is the bottleneck again'''

'Log1p or expm is the bottleneck again'

In [108]:
@numba.njit
def expm(data):
    expm = np.zeros_like(data)
    for i in range(0,len(data)):
        expm[i] = math.expm1(data[i])
    return expm

In [109]:
np.allclose(expm(data), np.expm1(data))

True

In [110]:
%timeit expm(data)

2.63 ms ± 23.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
%timeit np.expm1(data)

240 µs ± 4.16 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [112]:
%timeit expm(school)

13.1 µs ± 18.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [113]:
%timeit np.expm1(school)

13.3 µs ± 117 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [114]:
'''numpy is a bit faster
Lets see the overall performance'''

'numpy is a bit faster\nLets see the overall performance'

In [115]:
def _gpinv_new(probs, kappa, sigma):
    """Inverse Generalized Pareto distribution function."""
    # pylint: disable=unsupported-assignment-operation, invalid-unary-operand-type
    x = np.full_like(probs, np.nan)
    if sigma <= 0:
        return x
    ok = (probs > 0) & (probs < 1)
    if np.all(ok):
        if np.abs(kappa) < np.finfo(float).eps:
            x = -log_1p(-probs)
        else:
            x = np.expm1(-kappa * np.log1p(-probs)) / kappa
        x *= sigma
    else:
        if np.abs(kappa) < np.finfo(float).eps:
            x[ok] = -log_1p(-probs[ok])
        else:
            x[ok] = np.expm1(-kappa * np.log1p(-probs[ok])) / kappa
        x *= sigma
        x[probs == 0] = 0
        if kappa >= 0:
            x[probs == 1] = np.inf
        else:
            x[probs == 1] = -sigma / kappa
    return x

In [116]:
a = _gpinv_new(data,-5,2)

In [117]:
b = az.stats.stats._gpinv(data, -5, 2)

In [118]:
a

array([       nan,        nan,        nan, ..., 0.39939703,        nan,
              nan])

In [119]:
b


array([       nan,        nan,        nan, ..., 0.39939703,        nan,
              nan])

In [120]:
a-b

array([nan, nan, nan, ...,  0., nan, nan])

In [121]:
%timeit _gpinv_new(data, -5, 2)

2.72 ms ± 77 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [122]:
%timeit az.stats.stats._gpinv(data, -5, 2)

2.7 ms ± 37.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [123]:
%timeit _gpinv_new(school, -5, 2)

49.5 µs ± 409 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [124]:
%timeit az.stats.stats._gpinv(school, -5, 2)

49.8 µs ± 382 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [125]:
'''numba _gpinv is similar in performance'''

'numba _gpinv is similar in performance'

In [126]:
def generator(dataset_dict,ic='waic'):
    ic_i = "{}_i".format(ic)

    ics = pd.DataFrame()
    names = []
    for name, dataset in dataset_dict.items():
        names.append(name)
        ics = ics.append([waic(dataset, pointwise=True, scale="deviance")])
    ics.index = names
    ics.sort_values(by=ic, inplace=True, ascending=True)
    return ics, ic_i

In [127]:
ics, ic_i = generator({"1":load_arviz_data("centered_eight"),"2":load_arviz_data("non_centered_eight")})

In [128]:
ics

,waic,waic_se,p_waic,warning,waic_i,waic_scale
2,61.302151,2.727291,0.820067,0,"[9.732429123207623, 6.813708378182436, 7.70616...",deviance
1,61.429587,2.689941,0.919548,0,"[9.76127241619229, 6.832600316056821, 7.725554...",deviance


In [129]:
lp = LineProfiler()
wrapper = lp(_ic_matrix)
wrapper(ics,ic_i)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.002461 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: _ic_matrix at line 235

Line #      Hits         Time  Per Hit   % Time  Line Contents
   235                                           def _ic_matrix(ics, ic_i):
   236                                               """Store the previously computed pointwise predictive accuracy values (ics) in a 2D matrix."""
   237         1         21.0     21.0      0.9      cols, _ = ics.shape
   238         1        440.0    440.0     17.9      rows = len(ics[ic_i].iloc[0])
   239         1          9.0      9.0      0.4      ic_i_val = np.zeros((rows, cols))
   240                                           
   241         3        102.0     34.0      4.1      for idx, val in enumerate(ics.index):
   242         2       1853.0    926.5     75.3          ic = ics.loc[val][ic_i]
   243                                           
   244         2          7.0      3.5      0.3          if len(ic

In [130]:
@numba.jit
def loop_lifter(ics,rows,cols):
    ic_i_val = np.zeros((rows, cols))
    for idx, val in enumerate(ics.index):
        ic = ics.loc[val][ic_i]

        if len(ic) != rows:
            raise ValueError("The number of observations should be the same across all models")

        ic_i_val[:, idx] = ic
    return ic_i_val

def _ic_matrix_new(ics, ic_i):
    """Store the previously computed pointwise predictive accuracy values (ics) in a 2D matrix."""
    cols, _ = ics.shape
    rows = len(ics[ic_i].iloc[0])
    return rows, cols, loop_lifter(ics,rows,cols)


In [131]:
%timeit _ic_matrix_new(ics, ic_i)

854 µs ± 151 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [132]:
%timeit az.stats.stats._ic_matrix(ics, ic_i)

624 µs ± 5.58 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [133]:
'''Loop lifitng is not useful in this case. Numba fails at no nopython mode and falls back to pyobject mode.
   Original _ic_matrix is better'''

'Loop lifitng is not useful in this case. Numba fails at no nopython mode and falls back to pyobject mode.\n   Original _ic_matrix is better'

In [134]:
data = np.random.randn(1000,1000,20)
sample_stats = {"log_likelihood":data}
data = from_dict(sample_stats=sample_stats)
school = load_arviz_data("centered_eight")


In [135]:
lp = LineProfiler()
wrapper = lp(waic)
wrapper(data,True)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.645326 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: waic at line 868

Line #      Hits         Time  Per Hit   % Time  Line Contents
   868                                           def waic(data, pointwise=False, scale="deviance"):
   869                                               """Calculate the widely available information criterion.
   870                                           
   871                                               Also calculates the WAIC's standard error and the effective number of
   872                                               parameters of the samples in trace from model. Read more theory here - in
   873                                               a paper by some of the leading authorities on model selection
   874                                               dx.doi.org/10.1111/1467-9868.00353
   875                                           
   876                                          

/home/banzee/Desktop/arviz/arviz/stats/stats.py:934: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        
  """


In [136]:
def waic_new(data, pointwise=False, scale="deviance"):
    inference_data = convert_to_inference_data(data)
    for group in ("sample_stats",):
        if not hasattr(inference_data, group):
            raise TypeError(
                "Must be able to extract a {group} group from data!".format(group=group)
            )
    if "log_likelihood" not in inference_data.sample_stats:
        raise TypeError("Data must include log_likelihood in sample_stats")
    log_likelihood = inference_data.sample_stats.log_likelihood

    if scale.lower() == "deviance":
        scale_value = -2
    elif scale.lower() == "log":
        scale_value = 1
    elif scale.lower() == "negative_log":
        scale_value = -1
    else:
        raise TypeError('Valid scale values are "deviance", "log", "negative_log"')

    n_samples = log_likelihood.chain.size * log_likelihood.draw.size
    new_shape = (n_samples, np.product(log_likelihood.shape[2:]))
    log_likelihood = log_likelihood.values.reshape(*new_shape)

    lppd_i = _logsumexp(log_likelihood, axis=0, b_inv=log_likelihood.shape[0])

    vars_lpd = _var_2d(log_likelihood)
    warn_mg = 0
    if np.any(vars_lpd > 0.4):
        warnings.warn(
            """For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        """
        )
        warn_mg = 1

    waic_i = scale_value * (lppd_i - vars_lpd)
    waic_se = (len(waic_i) * _var_1d(waic_i)) ** 0.5
    waic_sum = np.sum(waic_i)
    p_waic = np.sum(vars_lpd)

    if pointwise:
        if np.equal(waic_sum, waic_i).all():  # pylint: disable=no-member
            warnings.warn(
                """The point-wise WAIC is the same with the sum WAIC, please double check
            the Observed RV in your model to make sure it returns element-wise logp.
            """
            )
        return pd.Series(
            data=[waic_sum, waic_se, p_waic, warn_mg, waic_i, scale],
            index=["waic", "waic_se", "p_waic", "warning", "waic_i", "waic_scale"],
        )
    else:
        return pd.Series(
            data=[waic_sum, waic_se, p_waic, warn_mg, scale],
            index=["waic", "waic_se", "p_waic", "warning", "waic_scale"],
        )

In [137]:
waic_new(data,True)

/home/banzee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        


waic                                                    19.9906
waic_se                                               0.0118671
p_waic                                                  19.9994
warning                                                       1
waic_i        [1.0020417664912404, 0.9963607756633119, 0.999...
waic_scale                                             deviance
dtype: object

In [138]:
az.stats.waic(data,True)

/home/banzee/Desktop/arviz/arviz/stats/stats.py:934: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        
  """


waic                                                    19.9906
waic_se                                               0.0118671
p_waic                                                  19.9994
warning                                                       1
waic_i        [1.0020417664912467, 0.9963607756633732, 0.999...
waic_scale                                             deviance
dtype: object

In [139]:
%timeit  waic_new(data,True)

/home/banzee/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        


586 ms ± 18.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [140]:
%timeit  az.stats.waic(data,True)

/home/banzee/Desktop/arviz/arviz/stats/stats.py:934: UserWarning: For one or more samples the posterior variance of the log predictive
        densities exceeds 0.4. This could be indication of WAIC starting to fail see
        http://arxiv.org/abs/1507.04544 for details
        
  """


641 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [141]:
%timeit  waic_new(school,True)

1.73 ms ± 61.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [142]:
%timeit  az.stats.waic(school,True)

2.01 ms ± 87.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [143]:
"""""""""""""""""""""""""""""""""""""""psislw"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

'psislw'

In [144]:
@numba.njit
def max_elem(data):
    max = data[0]
    for i in range(0,len(data)):
        if data[i]>max:
            max = data[i]
    return max

In [145]:
data = np.random.rand(1000,1000)

In [146]:
lp = LineProfiler()
wrapper = lp(psislw)
wrapper(data, 0.66)
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.77662 s
File: /home/banzee/Desktop/arviz/arviz/stats/stats.py
Function: psislw at line 420

Line #      Hits         Time  Per Hit   % Time  Line Contents
   420                                           def psislw(log_weights, reff=1.0):
   421                                               """
   422                                               Pareto smoothed importance sampling (PSIS).
   423                                           
   424                                               Parameters
   425                                               ----------
   426                                               log_weights : array
   427                                                   Array of size (n_samples, n_observations)
   428                                               reff : float
   429                                                   relative MCMC efficiency, `ess / n`
   430                                           
   431       

In [147]:
'''Lets use our own gpd funcs'''

'Lets use our own gpd funcs'

In [148]:
def psislw_new(log_weights, reff=1.0):
    rows, cols = log_weights.shape

    log_weights_out = np.copy(log_weights, order="F")
    kss = np.empty(cols)

    # precalculate constants
    cutoff_ind = -int(np.ceil(min(rows / 5.0, 3 * (rows / reff) ** 0.5))) - 1
    cutoffmin = np.log(np.finfo(float).tiny)  # pylint: disable=no-member, assignment-from-no-return
    k_min = 1.0 / 3

    # loop over sets of log weights
    for i, x in enumerate(log_weights_out.T):
        # improve numerical accuracy
        x -= np.max(x)
        # sort the array
        x_sort_ind = np.argsort(x)
        # divide log weights into body and right tail
        xcutoff = max(x[x_sort_ind[cutoff_ind]], cutoffmin)

        expxcutoff = np.exp(xcutoff)
        tailinds, = np.where(x > xcutoff)  # pylint: disable=unbalanced-tuple-unpacking
        x_tail = x[tailinds]
        tail_len = len(x_tail)
        if tail_len <= 4:
            # not enough tail samples for gpdfit
            k = np.inf
        else:
            # order of tail samples
            x_tail_si = np.argsort(x_tail)
            # fit generalized Pareto distribution to the right tail samples
            x_tail = np.exp(x_tail) - expxcutoff
            k, sigma = _gpdfit_new(x_tail[x_tail_si])

            if k >= k_min:
                # no smoothing if short tail or GPD fit failed
                # compute ordered statistic for the fit
                sti = np.arange(0.5, tail_len) / tail_len
                smoothed_tail = _gpinv_new(sti, k, sigma)
                smoothed_tail = np.log(  # pylint: disable=assignment-from-no-return
                    smoothed_tail + expxcutoff
                )
                # place the smoothed tail into the output array
                x[tailinds[x_tail_si]] = smoothed_tail
                # truncate smoothed values to the largest raw weight 0
                x[x > 0] = 0
        # renormalize weights
        x -= _logsumexp(x)
        # store tail index k
        kss[i] = k

    return log_weights_out, kss

In [149]:
l,k = psislw_new(data,0.84)

In [150]:
c,d = az.stats.psislw(data,0.84)

In [151]:
np.allclose(l,c)

True

In [152]:
np.allclose(k,d)

True

In [153]:
%timeit psislw_new(data,0.84)

547 ms ± 9.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [154]:
%timeit az.stats.psislw(data,0.84)

543 ms ± 4.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [155]:
'''Not much improvement'''

'Not much improvement'